## 🛠️ **ETL (Extract, Transform, Load)**



#### **📂 Procesamiento del 1er archivo: `Yelp/business.pkl`**

####  **Importamos las librerías que vamos a usar**


In [1]:
import pandas as pd
import data_utils   
from data_utils import data_type_check, data_type_check_pkl

import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import matplotlib.pyplot as plt


#### 📦 **Extraccion** de los datos y primera exploración 


Mostrar info como en datatypecheck
#data_type_check_pkl('../0_Dataset/Yelp/business.pkl')

**PENDIENTE**
% de nulos, .shape, .columns


___

In [2]:
business = pd.read_pickle('../0_Dataset/Yelp/business.pkl')
business.info()
business.sample(2)

<class 'pandas.core.frame.DataFrame'>
Index: 150346 entries, 0 to 150345
Data columns (total 28 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   business_id   150346 non-null  object
 1   name          150346 non-null  object
 2   address       150346 non-null  object
 3   city          150346 non-null  object
 4   state         150343 non-null  object
 5   postal_code   150346 non-null  object
 6   latitude      150346 non-null  object
 7   longitude     150346 non-null  object
 8   stars         150346 non-null  object
 9   review_count  150346 non-null  object
 10  is_open       150346 non-null  object
 11  attributes    136602 non-null  object
 12  categories    150243 non-null  object
 13  hours         127123 non-null  object
 14  business_id   5 non-null       object
 15  name          5 non-null       object
 16  address       5 non-null       object
 17  city          5 non-null       object
 18  state         5 non-null     

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
111132,tXj7zTjcSjCe13qJmgz4sA,Le Gnome,"West Edmonton Mall, 1814 8770-170 St",Edmonton,TN,T5T 4M2,53.522659,-113.617949,3.0,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76857,Y2mVo-tSjuj3VYzzKmvrqw,Starr Heating and Air Conditioning,,Philadelphia,PA,19123,39.964454,-75.145782,4.5,41,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Tenemos columnas duplicadas, procedemos a quitarlas.


In [3]:
business = business.loc[:, ~business.columns.duplicated()]
business.info()

<class 'pandas.core.frame.DataFrame'>
Index: 150346 entries, 0 to 150345
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   business_id   150346 non-null  object
 1   name          150346 non-null  object
 2   address       150346 non-null  object
 3   city          150346 non-null  object
 4   state         150343 non-null  object
 5   postal_code   150346 non-null  object
 6   latitude      150346 non-null  object
 7   longitude     150346 non-null  object
 8   stars         150346 non-null  object
 9   review_count  150346 non-null  object
 10  is_open       150346 non-null  object
 11  attributes    136602 non-null  object
 12  categories    150243 non-null  object
 13  hours         127123 non-null  object
dtypes: object(14)
memory usage: 17.2+ MB


#### 🔁 **TRANSFORM**

Nos interesa observar las categorías por lo que primero vamos a corroborar que no tenga duplicados

In [4]:
business.categories.value_counts()

categories
Beauty & Spas, Nail Salons                                                                                       1012
Restaurants, Pizza                                                                                                935
Nail Salons, Beauty & Spas                                                                                        934
Pizza, Restaurants                                                                                                823
Restaurants, Mexican                                                                                              728
                                                                                                                 ... 
Dermatologists, Health & Medical, Cosmetic Surgeons, Doctors, Acne Treatment, Skin Care, Beauty & Spas              1
Home Services, Home & Garden, Nurseries & Gardening, Hardware Stores, Shopping, Building Supplies, Appliances       1
Food Trucks, Smokehouse, Restaurants, Food, B

Verificamos los nulos y duplicados de categories

In [5]:
# Cuenta los duplicados y nulos en la columna 'categories' antes de cualquier eliminación
duplicados_inicial = business['categories'].duplicated().sum()
nulos_inicial = business['categories'].isna().sum()

print("Antes: Cantidad de duplicados en 'categories':", duplicados_inicial)
print("Antes: Cantidad de NaNs en 'categories':", nulos_inicial)

# Elimina las filas con valores NaN en la columna 'categories'
business = business.dropna(subset=['categories'])

# Cuenta los duplicados y nulos en la columna 'categories' después de eliminar NaNs
duplicados_despues_nulos = business['categories'].duplicated().sum()
nulos_despues_nulos = business['categories'].isna().sum()

print("Después: Cantidad de NaNs en 'categories' ", nulos_despues_nulos)


'''
________________________________________________________________
FUTURO: Descomentar o borrar
Elimina las filas duplicadas basadas en la columna 'categories'
business = business.drop_duplicates(subset=['categories'])  
_________________________________________________________________  
'''

# Cuenta los duplicados en la columna 'categories' después de eliminar duplicados
duplicados_final = business['categories'].duplicated().sum()

print("Después: Cantidad de duplicados en 'categories'", duplicados_final)


Antes: Cantidad de duplicados en 'categories': 67185
Antes: Cantidad de NaNs en 'categories': 103
Después: Cantidad de NaNs en 'categories'  0
Después: Cantidad de duplicados en 'categories' 67083


Dado que diferentes negocios pueden compartir las mismas 'categories', no borramos los  67083 duplicados 


### Filtramos pizzerias en el estado Florida


Vamos a filtrar por estado. Para eso primero observamos todos los estados y luego elejimos el que vamos a usar

In [6]:
business['state'].value_counts()

state
PA     34013
FL     26311
TN     12045
IN     11238
MO     10909
LA      9917
AZ      9907
NJ      8529
NV      7712
AB      5571
CA      5201
ID      4463
DE      2261
IL      2142
TX         4
CO         3
WA         2
HI         2
MA         2
NC         1
UT         1
MT         1
MI         1
SD         1
XMS        1
VI         1
VT         1
Name: count, dtype: int64

### Filtramos las categorías más frecuentes en general en Estados Unidos


In [7]:
# Obtener el conteo de cada categoría en el DataFrame completo
conteo_categorias_totales = business['categories'].value_counts()

# Obtener las categorías únicas y ordenarlas alfabéticamente
categorias_unicas_totales = sorted(business['categories'].unique())

# Calcular el top 10 de categorías más frecuentes en general
top_10_categorias_totales = conteo_categorias_totales.head(10)

# Imprimir el top 10 de categorías más frecuentes en general
print("Top 10 Categorías Más Frecuentes en General:")
for i, (categoria, conteo) in enumerate(top_10_categorias_totales.items(), 1):
    print(f"{i}. {categoria}: {conteo} veces")

# Si hay más categorías, mostrar un mensaje de resumen
if len(conteo_categorias_totales) > 10:
    print(f"\nY {len(conteo_categorias_totales) - 10} categorías más...")

print("\n")

Top 10 Categorías Más Frecuentes en General:
1. Beauty & Spas, Nail Salons: 1012 veces
2. Restaurants, Pizza: 935 veces
3. Nail Salons, Beauty & Spas: 934 veces
4. Pizza, Restaurants: 823 veces
5. Restaurants, Mexican: 728 veces
6. Restaurants, Chinese: 708 veces
7. Mexican, Restaurants: 672 veces
8. Chinese, Restaurants: 651 veces
9. Food, Coffee & Tea: 508 veces
10. Beauty & Spas, Hair Salons: 493 veces

Y 83150 categorías más...




In [8]:
'''
Unir categorias?
'''

'\nUnir categorias?\n'

### Vamos a observar las categorías relacionadas con restaurantes en Florida


In [9]:
# Filtrar el DataFrame original para obtener solo las categorías relacionadas con restaurantes en Florida
df_estado_FL = business[business['categories'].str.lower().str.contains('restaurant', na=False) & (business['state'] == 'FL')].reset_index(drop=True)

# Obtener el conteo de cada categoría específicamente en Florida
conteo_categorias_FL = df_estado_FL['categories'].value_counts()

# Calcular el top 10 de categorías más frecuentes en Florida
top_10_categorias_FL = conteo_categorias_FL.head(10)

# Imprimir el top 10 de categorías más frecuentes en Florida
print("Top 10 Categorías Más Frecuentes en Florida:")
for i, (categoria, conteo) in enumerate(top_10_categorias_FL.items(), 1):
    print(f"{i}. {categoria}: {conteo} veces")

# Si hay más categorías, mostrar un mensaje de resumen
if len(conteo_categorias_FL) > 10:
    print(f"\nY {len(conteo_categorias_FL) - 10} categorías más...")

Top 10 Categorías Más Frecuentes en Florida:
1. Restaurants, Pizza: 163 veces
2. Pizza, Restaurants: 142 veces
3. Mexican, Restaurants: 122 veces
4. Restaurants, Chinese: 121 veces
5. Restaurants, Mexican: 118 veces
6. Chinese, Restaurants: 108 veces
7. Restaurants, Italian: 61 veces
8. Italian, Restaurants: 54 veces
9. American (New), Restaurants: 42 veces
10. Restaurants, American (New): 40 veces

Y 6468 categorías más...


### Filtramos por la el interes del proyecto: Pizzerias

Primero vamos a contar la cantidad de Pizzerias por estado


In [10]:
# Asegurar que 'categories' es de tipo string
business['categories'] = business['categories'].astype(str)

# Filtra las filas que contienen 'pizza' en 'categories'
pizzerias = business[business['categories'].str.lower().str.contains('pizza', na=False)]

# Cuenta la cantidad de pizzerías en cada estado
conteo_pizzerias_por_estado = pizzerias['state'].value_counts()

# Imprime el resultado
print("Cantidad de Pizzerías por Estado:")
print(conteo_pizzerias_por_estado)


Cantidad de Pizzerías por Estado:
state
PA    1554
FL    1261
TN     559
IN     510
AZ     499
MO     485
LA     475
NJ     399
NV     363
AB     279
CA     272
ID     225
IL     108
DE     103
WA       1
Name: count, dtype: int64


### Filtramos las pizzerias de Florida


In [13]:
#Filtrar el DataFrame original para obtener solo las categorías relacionadas con restaurantes en Florida
#Solo elegimos los locales abiertos
df_estado_FL = business[business['categories'].str.lower().str.contains('pizza', na=False) & (business['state'] == 'FL') & (business['is_open'] == 1)].reset_index(drop=True)
data_type_check(df_estado_FL)
df_estado_FL.sample(2)



 Resumen del dataframe:

Dimensiones:  (883, 14)
         columna  %_no_nulos  %_nulos  total_nulos tipo_dato
0    business_id      100.00     0.00            0    object
1           name      100.00     0.00            0    object
2        address      100.00     0.00            0    object
3           city      100.00     0.00            0    object
4          state      100.00     0.00            0    object
5    postal_code      100.00     0.00            0    object
6       latitude      100.00     0.00            0    object
7      longitude      100.00     0.00            0    object
8          stars      100.00     0.00            0    object
9   review_count      100.00     0.00            0    object
10       is_open      100.00     0.00            0    object
11    attributes       98.98     1.02            9    object
12    categories      100.00     0.00            0    object
13         hours       88.90    11.10           98    object


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
127,-jkc-0YYvwkBaNnNyiUHNg,Hidden Pizza,345 N Virginia St,Reno,FL,89501,39.528789,-119.814354,4.0,115,1,"{'BusinessParking': '{'garage': True, 'street'...","Restaurants, Pizza, Italian","{'Monday': '0:0-0:0', 'Tuesday': '17:0-1:0', '..."
185,u-pogbWQ-Hwl6qTQjVIAOQ,Pizza@ Tyrone Square,6901 22nd Ave N,Saint Petersburg,FL,33710,27.793023,-82.733725,4.5,32,1,"{'WheelchairAccessible': 'True', 'RestaurantsP...","Pizza, Restaurants, Fast Food","{'Monday': '10:0-21:0', 'Tuesday': '10:0-21:0'..."


In [14]:
# contar valores unicos en state
df_estado_FL.state.value_counts()



state
FL    883
Name: count, dtype: int64

#### **📤 LOAD**

In [ ]:
#guardar en parquet